In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key


file = "Weatherpy_df.csv"
df_main = pd.read_csv(file)

df_main

# Humidity Heatmap 
gmaps.configure(api_key=g_key)
locations = df_main[["Latitude", "Longitude"]].astype(float)
Humid = df_main["Humidity(%)"].astype(float)
fig = gmaps.figure(center = (0,0),zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=Humid, dissipating = False)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
# Filtering dataframe to 70 < temp < 80 and windspeed <10 and saving it to df_main2
df_main2 = df_main[(df_main["Max_Temp(F)"] < 80) & (df_main["Max_Temp(F)"] > 70) & (df_main["Wind Speed(mph)"] < 10) & (df_main["Cloudiness(%)"] == 0)]
df_main2 = df_main2.reset_index()
df_main2

,index,Unnamed: 0,City,Latitude,Longitude,Max_Temp(F),Humidity(%),Cloudiness(%),Wind Speed(mph)
0,4,4,Tchollire,8.40,14.17,74.93,25,0,0.69
1,40,40,Kisangani,0.52,25.20,71.33,79,0,4.36
2,52,52,Lanquin,15.57,-89.97,76.73,59,0,5.08
3,86,86,Kuruman,-27.45,23.43,76.73,64,0,7.85
4,212,212,Faranah,10.04,-10.74,73.13,24,0,2.82
5,222,222,Gorom-Gorom,14.44,-0.24,76.73,22,0,6.02
6,232,232,Bandarbeyla,9.49,50.81,76.73,74,0,9.73
7,370,370,Chokwe,-24.53,32.98,73.13,92,0,4.65
8,430,430,Ahipara,-35.17,173.17,78.53,57,0,1.99


In [5]:
#hotel_df creation using Google API

hotel_df = pd.DataFrame(columns = ["City","Lat","Lon","Hotel Name","Country"])

count = 0
for n in np.arange(0,len(df_main2["City"])):
    # geocoordinates
    target_coordinates = str(df_main2["Latitude"][n]) +","+ str(df_main2["Longitude"][n])
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()

    try :
        for n2 in np.arange(0,len(response["results"])):
            new_row =[]
            new_row.append(df_main["City"][n])   
            new_row.append(response["results"][n2]["geometry"]["location"]["lat"])
            new_row.append(response["results"][n2]["geometry"]["location"]["lng"])
            new_row.append(response["results"][n2]["name"])
            new_row.append(response["results"][n2]["plus_code"]["compound_code"])
            hotel_df.loc[count] = new_row
            count = count + 1
    except:
        print("City Not Found. Skipping...")
        
        

City Not Found. Skipping...


In [6]:
hotel_df        

,City,Lat,Lon,Hotel Name,Country
0,Constitucion,8.402334,14.170747,"Auberge ""l'amicale""","C52C+W7 Tcholliré, Cameroon"
1,Gizo,0.511962,25.172548,Grand Ramela Hotel,"G56F+Q2 Kisangani, Congo - Kinshasa"
2,Gizo,0.521491,25.183067,Hotel Kisangani,"G5CM+H6 Kisangani, Congo - Kinshasa"
3,Gizo,0.517757,25.179076,Ramela Hotel,"G59H+4J Kisangani, Congo - Kinshasa"
4,Gizo,0.519013,25.198962,Hotel Du Bonheur,"G59X+JH Kisangani, Congo - Kinshasa"
...,...,...,...,...,...
77,Mahon,-35.162681,173.154928,Beach Abode Beachfront Lodge,"R5P3+WX Ahipara, Northland, New Zealand"
78,Mahon,-35.163687,173.154090,Ahipara Tides,"R5P3+GJ Ahipara, Northland, New Zealand"
79,Mahon,-35.175451,173.139506,Ahipara Bay Motel,"R4FQ+RR Ahipara, Northland, New Zealand"
80,Mahon,-35.163472,173.155268,Ahipara Sunset Accommodation,"R5P4+J4 Ahipara, Northland, New Zealand"


In [9]:
for n in np.arange(0,len(hotel_df["Country"])):
    hotel_df["Country"][n] = hotel_df["Country"][n][hotel_df["Country"][n].find(',')+1 : ]
    hotel_df["Country"][n] = hotel_df["Country"][n][hotel_df["Country"][n].find(',')+1 : ]

C:\Users\jaunt\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jaunt\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:

# Create bank symbol layer

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]].astype(float)

hotel_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)

#info_box_content=[f"Hotel Name: {hotel}" for hotel in hotel_df["Hotel Name"]]

fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))